In [1]:
# 

In [21]:
from collections.abc import Iterable
import pandas as pd
import re
from chemicals import *
import convert

compounds = ['CO2', 'N2', 'CH4', 'C2H6', 'C3H8', 'C4H10']
mole_fraction = [0.12, 0.021, 0.8, 0.042, 0.013, 0.004]
weight_fraction = [0.12, 0.021, 0.8, 0.042, 0.013, 0.004]
MOLAR_VOLUME = 379.3  #scf/lbmole


ModuleNotFoundError: No module named 'chemicals'

In [2]:
class Emission:
    
    def __init__(self, compounds:Iterable[str], composition:Iterable[float], HC, FUEL_EFFICENCY, *args, **kwargs):
        self.compounds = compounds
        self.composition = composition
        self.HC = HC
        self.FUEL_EFFICIENCY = FUEL_EFFICENCY
        self.args = args
        self.kwargs = kwargs
        
    def get_molecular_weight_list(self):
        '''This function returns the list of molecular weight for the components in a gas mixture.
            The function Receives an array or list of the common name or molecular formular of the gas components as argument.
        '''
        # list of gas components.

        list_of_components = []
        list_of_molecular_weight = []
        list_of_molecular_formula = []
        for i in identifiers.pubchem_db:
            for j in self.compounds:
                if i.formula == j or i.common_name == j:
                    list_of_components.append(i.common_name)
                    list_of_molecular_weight.append(i.MW)
                    list_of_molecular_formula.append(i.formula)
                else:
                    None

        # Create a pandas dataframe from three lists of column names Compound, Formula, and Molecular weight.
        data = {
            'Compound': list_of_components,
            'Formula': list_of_molecular_formula,
            'Molecular Weight': list_of_molecular_weight
        }
        df = pd.DataFrame(data)

        return df
        
    def molecularweight_mixture_molefraction(self, molecular_weight:Iterable[float]):
        '''This function calculates molecular weight fraction of the gas mixture given:
        mole fraction, and molecular weight

        '''
        x = zip(self.composition, molecular_weight)
        product_list = []
        for a, b in x:
            product_list.append(a*b)

        return sum(product_list)
    
    def molecularweight_mixture_weightfraction(self, molecular_weight:Iterable[float]):
        '''This function calculates molecular weight fraction of the gas mixture given:
        component weight fractions, and molecular weights.

        '''
        x = zip(self.composition, molecular_weight)
        product_list = []
        for a, b in x:
            product_list.append(a/b)

        return sum(product_list)
    
    def find_hydrocarbons_in_gas_mixture(self):
        '''
        This function returns only the hydrocarbon components present in a mixture of gases.
        '''
        # create a regular expression in python for strings that contains on C and H, and digits between the C and H
        pattern = r'^[CH\d]+$'
        list_of_hydrocarbons = []
        for i in identifiers.pubchem_db:
            if re.search(pattern, i.formula):
                for j in self.compound:
                    if i.formula == j or i.common_name == j:
                        list_of_hydrocarbons.append(i)
            else:
                None
        return list_of_hydrocarbons
    
    def adjusted_composition(self):
        '''This function takes the mole fraction or weight fraction of the gas mixture as argument and returns a list of the adjusted mole fraction of weight fraction.'''

        # combine two lists, compound and composition into a dictionary of key and value pair.
        dictionary = dict(zip(self.compounds, self.composition))
        included_list = self.find_hydrocarbons_in_gas_mixture(self.compound)
        for key, value in dictionary.items():
            if key in [i.formula for i in included_list]:
                dictionary[key] = value  # Set value to value if key is in include_list
            else:
                dictionary[key] = 0 # Set value to value if key is not in include_list

        # find the sum of the new dictionary values when non-hydrocarbon components are set to zero
        sum_composition = sum(dictionary.values())
        for key, values in dictionary.items():
            dictionary[key] = values/sum_composition
        return dictionary
    
    def carbon_ratio(compound):
        matches = re.findall(r"C(\d+)H", compound)
        if compound == 'CH4':
            digits = 1
        elif matches == []:
            digits = None
        else:
            digits = matches[0]
        return digits
    
    def adjustedComposition_molecularweight_table(self):
        
        # List to define the sorting order
        sorting_order = list(self.adjusted_composition(self.compounds, self.composition))

        # Sort the DataFrame based on the sorting_order list
        df_sorted = self.get_molecular_weight_list(self.compounds).sort_values(by='Formula', key=lambda x: x.map({value: i for i, value in enumerate(sorting_order)})).reset_index(drop=True)

        # Append adjusted composition to sorted dataFrame
        df_sorted['Adjusted Composition'] = pd.Series(self.adjusted_composition(self.compounds, self.composition).values())


        # Append Carbon Ratio to the dataframe
        df_sorted['Carbon Ratio'] = df_sorted['Formula'].apply(lambda formula: carbon_ratio(formula))
        return df_sorted.fillna(0)
    
    def carbon_content_mole_basis(self):
        data_table = self.adjustedComposition_molecularweight_table(self.compounds, self.composition)
        data_table['Mole Fraction'] = pd.Series(self.composition)
        data_table = data_table.fillna(0)
        mole_fraction_carbonratio = lambda row: row['Mole Fraction']*float(row['Carbon Ratio'])

        data_table['Carbon Content'] = data_table.apply(mole_fraction_carbonratio, axis=1)

        return data_table
    
    def carbon_content_weight_basis(self):
        data_table = self.adjustedComposition_molecularweight_table(self.compounds, self.composition)
        data_table['Weight Fraction'] = pd.Series(self.composition)
        data_table = data_table.fillna(0)
        mole_fraction_carbonratio = lambda row: row['Adjusted Composition']*float(row['Carbon Ratio'])*12/row['Molecular Weight']

        data_table['Carbon Content'] = data_table.apply(mole_fraction_carbonratio, axis=1)

        return data_table
    
    def carbon_content_fuel_mixture_mole_basis(self):
        C_mixture = sum(self.carbon_content_mole_basis()['Carbon Content'])
        return C_mixture

    def carbon_content_fuel_mixture_weight_basis(self):
        C_mixture = sum(self.carbon_content_weight_basis()['Carbon Content'])
        return C_mixture
        
    def CO2_composition(self):
        '''Mass of CO2 in the flared stream based on CO2 composition of the inlet stream'''
        dictionary = dict(zip(self.compounds, self.composition))
        co2_composition = dictionary['CO2']
        return co2_composition

    def CH4_composition(self):
        '''CH4 composition'''
        dictionary = dict(zip(self.compound, self.composition))
        CH4_composition = dictionary['CH4']
        return CH4_composition


In [3]:
class CO2(Emission):
        
    def inletvolume_molefraction(self):
        '''
        This function calculates the Carbon dioxide emission rate in tonnes when the flare inlet data is known.
        CALCULATION STEPS:
        '''
        CO2_mass_emission_rate = convert.lb_tonnes(convert.CO2_lbmole_lb(convert.scf_lbmole(self.HC)*(self.carbon_content_fuel_mixture_mole_basis()*self.FUEL_EFFICIENCY + self.CO2_composition(self.compounds, self.composition))))
        return CO2_mass_emission_rate
    
    def inletmass_weightfraction(self):
        '''
        This function calculates the Carbon dioxide emission rate in tonnes when the flare inlet data is known.
        CALCULATION STEPS:
        '''
        CO2_mass_emission_rate = convert.lb_tonnes(self.HC*(self.carbon_content_fuel_mixture_weight_basis()*self.FUEL_EFFICIENCY*44/12 + self.CO2_composition(self.compounds, self.composition)))
        return CO2_mass_emission_rate
    
    def outletvolume_molefraction(self):
        '''
        This function calculates the Carbon dioxide emission rate in tonnes when the flare outlet data is known.
        CALCULATION STEPS:
        '''
        CO2_mass_emission_rate = convert.lb_tonnes(convert.CO2_lbmole_lb(convert.scf_lbmole(self.HC)*(self.carbon_content_fuel_mixture_mole_basis()*(self.FUEL_EFFICIENCY/(1-self.FUEL_EFFICIENCY)) + CO2_composition(self.compounds, self.composition))))
        return CO2_mass_emission_rate
    
    def outletmass_weightfraction(self):
        '''
        This function calculates the Carbon dioxide emission rate in tonnes when the flare outlet data is known.
        CALCULATION STEPS:
        '''
        CO2_mass_emission_rate = convert.lb_tonnes(self.HC*(self.carbon_content_fuel_mixture_weight_basis()*(self.FUEL_EFFICIENCY/(1-self.FUEL_EFFICIENCY))*44/12 + self.CO2_composition(self.compounds, self.composition)))
        return CO2_mass_emission_rate

In [4]:
class CH4(Emission):
    
    def inletvolume_molefraction(self):
        '''
        This function calculates the Methane emission rate in tonnes when the flare inlet data is known.
        CALCULATION STEPS:
        '''
        CH4_mass_emission_rate = convert.lb_tonnes(convert.scf_lbmole(self.CH4_composition(self.compounds, self.composition)*self.HC)*(1-self.FUEL_EFFICIENCY)*16)
        return CH4_mass_emission_rate
    
    def inletmass_weightfraction(self):
        '''
        This function calculates the Methane emission rate in tonnes when the flare inlet data is known.
        CALCULATION STEPS:
        '''
        CH4_mass_emission_rate = convert.lb_tonnes(self.CH4_composition(self.compounds, self.composition)*self.HC*(1-self.FUEL_EFFICIENCY))
        return CH4_mass_emission_rate
    
    def outletvolume_molefraction(self):
        '''
        This function calculates the Methane emission rate in tonnes when the flare inlet data is known.
        CALCULATION STEPS:
        '''
        CH4_mass_emission_rate = convert.lb_tonnes(convert.scf_lbmole(self.CH4_composition(self.compounds, self.composition)*self.HC)*16)
        return CH4_mass_emission_rate
    
    def outletmass_weightfraction(self):
        '''
        This function calculates the Methane emission rate in tonnes when the flare inlet data is known.
        CALCULATION STEPS:
        '''
        CH4_mass_emission_rate = convert.lb_tonnes(self.CH4_composition(self.compounds, self.composition)*self.HC)
        return CH4_mass_emission_rate

In [7]:
'''
Pint is a Python package to define, operate and manipulate physical quantities: 
the product of a numerical value and a unit of measurement. 
It allows arithmetic operations between them and conversions from and to different units.

Install pint using:
conda install -c conda-forge pint
or 
pip install pint
'''
import pint
ureg = pint.UnitRegistry()
Q_ = ureg.Quantity

In [19]:
import convert

